In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib_venn import venn2 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Comparison MS data v1

In [ ]:
dataset1 = "deybis_05" # "deybis_05", "vanessa_05", "mentos_05", "Douban Online_Offline", "ACM_DBLP" # args.dataset
dataset2 = "vanessa_05" # "deybis_05", "vanessa_05", "mentos_05", "Douban Online_Offline", "ACM_DBLP" # args.dataset

encoder = "GIN"

In [ ]:
list_train_set = [
	["FrescoAmazonas_1", "FrescoAmazonas_2"],
	["FrescoCusco_1", "FrescoCusco_2"],
	["FrescoSanMartin_1", "FrescoSanMartin_2"],
	["SecoAmazonas_1", "SecoAmazonas_2"],
	["SecoCusco_1", "SecoCusco_2"],
	["SecoSanMartin_1", "SecoSanMartin_2"]
]

In [ ]:
tol_rt, tol_mz = 0.5, 15 # 0.5 min, 10 ppm

for train_set in list_train_set:
	df_1 = pd.read_csv(f"data/{dataset1}/output/node_alignment_{encoder}_{dataset1}_{train_set}.csv" , sep=";", decimal=",")
	df_2 = pd.read_csv(f"data/{dataset2}/output/node_alignment_{encoder}_{dataset2}_{train_set}.csv" , sep=";", decimal=",")

	df_match = (
		df_1.assign(key=1)
			.merge(df_2.assign(key=1), on="key", suffixes=("_1", "_2"))
			.drop(columns="key")
	)

	df_match["mz_error"] = (
		(df_match["Average Mz_1"] - df_match["Average Mz_2"]).abs() /
		((df_match["Average Mz_1"] + df_match["Average Mz_2"]) / 2)
	) * 1e6

	df_match["rt_error"] = (df_match["Average Rt_1"] - df_match["Average Rt_2"]).abs()
	df_match

	df_match_filter = df_match[df_match["mz_error"] < tol_mz]
	# df_match_filter = df_match[(df_match["mz_error"] < tol_mz) & (df_match["rt_error"] < tol_rt)]
	
	print(train_set)
	display(df_match_filter)

In [ ]:
tol_rt, tol_mz = 0.5, 15 # 0.5 min, 10 ppm

df_1 = pd.read_csv(f"data/{dataset1}/raw.csv", index_col=0).iloc[:, :3] # Sep
df_2 = pd.read_csv(f"data/{dataset2}/raw.csv", index_col=0).iloc[:, :3] # Dec
print(len(df_1), len(df_2))

df_match = (
	df_1.assign(key=1)
		.merge(df_2.assign(key=1), on="key", suffixes=("_1", "_2"))
		.drop(columns="key")
)

df_match["mz_error"] = (
	(df_match["Average Mz_1"] - df_match["Average Mz_2"]).abs() /
	((df_match["Average Mz_1"] + df_match["Average Mz_2"]) / 2)
) * 1e6

df_match["rt_error"] = (df_match["Average Rt_1"] - df_match["Average Rt_2"]).abs()
df_match

In [ ]:
df_match_filter = df_match[df_match["mz_error"] < tol_mz]
df_match_filter

In [ ]:
df_match_filter = df_match[(df_match["mz_error"] < tol_mz) & (df_match["rt_error"] < tol_rt)]
df_match_filter

In [ ]:
""" rt_diff = (df_match["Average Rt_1"] - df_match["Average Rt_2"]).abs()
mz_diff = (
	((df_match["Average Mz_1"] - df_match["Average Mz_2"]).abs() / (df_match["Average Mz_1"] + df_match["Average Mz_2"]) / 2)
) * 1e6
mask = (rt_diff <= tol_rt) & (mz_diff <= tol_mz)
# mask = (mz_diff <= tol_mz)

df_intersection = df_match.loc[mask].reset_index(drop=True)
df_intersection """

In [ ]:
""" tol_rt, tol_mz = 0.5, 0.01 # Change

for train_set in list_train_set:
	df_1 = pd.read_csv(f"data/{dataset1}/output/node_alignment_{encoder}_{dataset1}_{train_set}.csv" , sep=";", decimal=",")
	df_2 = pd.read_csv(f"data/{dataset2}/output/node_alignment_{encoder}_{dataset2}_{train_set}.csv" , sep=";", decimal=",")

	df_match = (
		df_1.assign(key=1)
			.merge(df_2.assign(key=1), on="key", suffixes=("_1", "_2"))
			.drop(columns="key")
	)

	rt_diff = (df_match["Average Rt_1"] - df_match["Average Rt_2"]).abs()
	mz_diff = (
		(df_match["Average Mz_1"] - df_match["Average Mz_2"]).abs() # / df_match["Average Mz_2"] * 1e6
	)
	# mask = (rt_diff <= tol_rt) & (mz_diff <= tol_mz)
	mask = (mz_diff <= tol_mz)

	df_intersection = df_match.loc[mask].reset_index(drop=True)
	print(train_set)
	display(df_intersection) """

### Comparison filter Deybis

In [ ]:
dataset1 = "deybis_filter_september_2br_10ar" # "deybis_filter_september_05", "deybis_filter_december_05", "deybis_05", "vanessa_05", "mentos_05", "Douban Online_Offline", "ACM_DBLP" # args.dataset
dataset2 = "deybis_filter_december_2br_10ar" # "deybis_filter_september_05", "deybis_filter_december_05",

# Parameters data
file = open(f"data/{dataset1}/input/parameters.json")
params = json.load(file)

methods = params["methods"]
print("Methods:\t", methods)

data_variations = params["data_variations"]
print("Data variations:", data_variations)

has_transformation = params["has_transformation"]
print("Has transformation:", has_transformation)

dimension = params["dimension"]
print("Dimension:\t", dimension)

threshold_corr = params["threshold_corr"]
print("Threshold corr:\t", threshold_corr)

iterations = params["iterations"]
print("Iterations:\t", iterations)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

groups_id_no = params["groups_id_no"]
print("Groups id (no):\t", groups_id_no)

seeds = params["seeds"]
print("Seeds:\t\t", seeds)

sensitivity = params["sensitivity"]
print("Sensitivity:\t", sensitivity)

In [ ]:
# Read raw data

df_join_raw = pd.read_csv("data/{}/input/raw.csv".format(dataset1), index_col=0)
df_join_raw

In [ ]:
# Filter

list_node_intersection = []
for group_id in groups_id:
	print(group_id)

	# Read common node
	common_node_id1 = sorted(np.load(f"data/{dataset1}/output/common_node/{group_id}.npy"))
	common_node_id2 = sorted(np.load(f"data/{dataset2}/output/common_node/{group_id}.npy"))
	# print(common_node_id1)
	# print(common_node_id2)

	intersection = sorted(list(set(common_node_id1) & set(common_node_id2)))
	# print(intersection)
	
	list_node_intersection.append([group_id, intersection])
	# list_node_intersection.append([group_id, ",".join(map(str, intersection))])

	# Plot
	plt.figure()
	venn2(subsets=(len(common_node_id1), len(common_node_id2), len(intersection)), set_labels=("September", "December"))
	plt.show()

df_comparison = pd.DataFrame(list_node_intersection, columns=["Group id", "Common node id"])
df_comparison

In [ ]:
# Filter by gropu id

for i, group_id in enumerate(groups_id):
	print(group_id)
	common_node_id = list_node_intersection[i][1]
	print(len(common_node_id), common_node_id)

	# Filter raw data
	df_join_raw_filter = df_join_raw.loc[common_node_id].iloc[:, [0, 1, 2]]
	# df_join_raw_filter.to_csv(f"data/{dataset}/output/filter_raw/{group_id}.csv", sep=";", decimal=",", index_label="Id")
	display(df_join_raw_filter)

In [ ]:
# Comparison with Statistics

df_sta = pd.read_csv(f"data/deybis_filter_september_2br_3ar/input/sta.csv") # Prof. Alfredo
df_sta

In [ ]:
# Filter by gropu id (using intersection nodes)

tol_rt, tol_mz = 0.5, 0.01
for i, group_id in enumerate(groups_id):
	print(group_id)
	common_node_id = list_node_intersection[i][1]
	print(common_node_id)

	# Filter raw data
	df_join_raw_filter = df_join_raw.loc[common_node_id].iloc[:, [0, 1]]

	df_1 = df_sta.copy()
	df_2 = df_join_raw_filter.copy()
	print(len(df_1), len(df_2))

	df_match = (
		df_1.assign(key=1)
			.merge(df_2.assign(key=1), on="key", suffixes=("_1", "_2"))
			.drop(columns="key")
	)

	df_match["mz_error"] = (
		(df_match["Average Mz_1"] - df_match["Average Mz_2"]).abs()
	)

	df_match["rt_error"] = (df_match["Average Rt_1"] - df_match["Average Rt_2"]).abs()
	# df_match_filter = df_match[df_match["mz_error"] < tol_mz]
	df_match_filter = df_match[(df_match["mz_error"] < tol_mz) & (df_match["rt_error"] < tol_rt)]
	display(df_match_filter)

In [ ]:
# Filter by gropu id (using common nodes)

dataset = dataset2 # Change, dataset1, dataset2
tol_rt, tol_mz = 0.5, 0.01
print(dataset)

for i, group_id in enumerate(groups_id):
	print(group_id)
	common_node_id = sorted(np.load(f"data/{dataset}/output/common_node/{group_id}.npy"))
	print(common_node_id)

	# Filter raw data
	df_join_raw_filter = df_join_raw.loc[common_node_id].iloc[:, [0, 1]]

	df_1 = df_sta.copy()
	df_2 = df_join_raw_filter.copy()
	print(len(df_1), len(df_2))

	df_match = (
		df_1.assign(key=1)
			.merge(df_2.assign(key=1), on="key", suffixes=("_1", "_2"))
			.drop(columns="key")
	)

	df_match["mz_error"] = (
		(df_match["Average Mz_1"] - df_match["Average Mz_2"]).abs()
	)

	df_match["rt_error"] = (df_match["Average Rt_1"] - df_match["Average Rt_2"]).abs()
	# df_match_filter = df_match[df_match["mz_error"] < tol_mz]
	df_match_filter = df_match[(df_match["mz_error"] < tol_mz) & (df_match["rt_error"] < tol_rt)]
	display(df_match_filter)

### Classification

In [25]:
# file = open("exp.json")
# experiment = json.load(file)
exp = "exp1" # experiment["exp"] # Change to static

file = open("experiments/output/{}/parameters.json".format(exp))
params = json.load(file)

print("Exp:\t\t", exp)

raw_data_file = params["raw_data_file"]
print("Raw data:\t", raw_data_file)

methods = params["methods"]
print("Methods:\t", methods)

has_transformation = params["has_transformation"]
print("Has transformation:", has_transformation)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

cuda = params["cuda"]
print("Cuda:\t", cuda)

Exp:		 exp1
Raw data:	 deybis_filter_september_2br_3ar_format
Methods:	 ['t-gae']
Has transformation: False
Dimension:	 32
Groups id:	 ['SecoAmazonas', 'FrescoSanMartin', 'FrescoCusco', 'FrescoAmazonas', 'SecoSanMartin', 'SecoCusco']
Subgroups id:	 {'SecoAmazonas': ['1', '2'], 'FrescoSanMartin': ['1', '2'], 'FrescoCusco': ['1', '2'], 'FrescoAmazonas': ['1', '2'], 'SecoSanMartin': ['1', '2'], 'SecoCusco': ['1', '2']}
Cuda:	 1


In [26]:
# df_join_raw = pd.read_csv(f"experiments/input/{exp}_raw.csv", index_col=0)
df_raw = pd.read_csv(f"experiments/raw_data/deybis_filter_september_all_format.csv", delimiter="|")
df_raw

,Alignment ID,Average Rt,Average Mz,Metabolite name,SecoAmazonas_1.1,SecoAmazonas_1.2,SecoAmazonas_1.3,SecoAmazonas_1.4,SecoAmazonas_1.5,SecoAmazonas_1.6,...,FrescoAmazonas_3.1,FrescoAmazonas_3.2,FrescoAmazonas_3.3,FrescoAmazonas_3.4,FrescoAmazonas_3.5,FrescoAmazonas_3.6,FrescoAmazonas_3.7,FrescoAmazonas_3.8,FrescoAmazonas_3.9,FrescoAmazonas_3.10
0,0,1.754,167.01318,NaN,2.102369e+07,5.191424e+06,2.297892e+06,2.022352e+06,1.922735e+06,1.522363e+06,...,2.691737e+07,2.667536e+07,3.425704e+06,3.852726e+06,4.906125e+06,4.632700e+06,6.474274e+06,5.309377e+06,1.222893e+07,1.096223e+07
1,1,2.120,152.05702,NaN,2.420068e+06,2.307995e+06,5.114995e+06,5.131455e+06,5.336355e+06,5.418774e+06,...,7.975709e+05,8.330269e+05,1.464225e+06,1.471219e+06,1.590734e+06,1.752319e+06,1.199727e+06,1.253488e+06,1.451674e+06,1.511296e+06
2,2,2.125,257.96816,NaN,9.001219e+06,9.873734e+06,7.146516e+05,6.834720e+05,7.314678e+06,7.242087e+06,...,1.464372e+07,1.561670e+07,1.069788e+06,1.134695e+06,1.007142e+07,9.805008e+06,1.503011e+07,1.535597e+07,1.626041e+07,1.720745e+07
3,3,2.128,207.98572,NaN,1.938034e+07,2.008965e+07,2.191925e+06,2.174522e+06,5.848345e+06,5.715126e+06,...,2.772725e+07,2.676637e+07,2.053523e+06,2.105461e+06,8.143728e+06,7.678579e+06,1.483699e+07,1.450835e+07,1.945829e+07,1.910929e+07
4,4,2.133,217.96001,NaN,9.418357e+06,9.172249e+06,1.434368e+07,1.480194e+07,1.936935e+07,1.923575e+07,...,1.372708e+07,1.383002e+07,2.095277e+07,2.200807e+07,2.830071e+07,2.649514e+07,2.400543e+07,2.153438e+07,2.538244e+07,2.781277e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,157,20.487,501.26053,NaN,5.709668e+06,5.691558e+06,4.908186e+06,5.048013e+06,5.715906e+06,5.190272e+06,...,7.366804e+06,6.834722e+06,6.712807e+06,6.186079e+06,7.730475e+06,7.233852e+06,7.394969e+06,6.816898e+06,7.023647e+06,7.784816e+06
158,158,22.679,520.34109,NaN,1.034447e+07,1.153674e+07,3.173041e+07,2.880942e+07,2.625058e+07,2.696883e+07,...,2.220213e+05,1.821823e+05,9.847894e+05,8.646981e+05,7.707464e+05,7.362852e+05,5.459844e+05,5.837284e+05,6.648591e+05,6.099219e+05
159,159,23.180,200.20136,NaN,7.190209e+06,6.419463e+06,1.939271e+07,2.036405e+07,2.358837e+07,2.184794e+07,...,9.140713e+06,9.683758e+06,3.105293e+07,2.459284e+07,3.208167e+07,3.071925e+07,3.133476e+07,3.285109e+07,4.688143e+07,4.851697e+07
160,160,24.234,496.34082,NaN,2.836296e+07,2.610888e+07,5.500906e+07,5.810079e+07,3.126886e+07,3.067800e+07,...,3.224507e+07,2.965402e+07,4.949670e+07,4.681355e+07,2.604815e+07,2.548679e+07,1.401562e+07,1.510820e+07,1.913652e+07,1.905453e+07


In [27]:
# Concat
df_join_raw = pd.concat([df_raw.iloc[:, :]], axis=1)
df_join_raw.set_index("Alignment ID", inplace=True)
df_join_raw = df_join_raw.rename_axis(None)
df_join_raw

,Average Rt,Average Mz,Metabolite name,SecoAmazonas_1.1,SecoAmazonas_1.2,SecoAmazonas_1.3,SecoAmazonas_1.4,SecoAmazonas_1.5,SecoAmazonas_1.6,SecoAmazonas_1.7,...,FrescoAmazonas_3.1,FrescoAmazonas_3.2,FrescoAmazonas_3.3,FrescoAmazonas_3.4,FrescoAmazonas_3.5,FrescoAmazonas_3.6,FrescoAmazonas_3.7,FrescoAmazonas_3.8,FrescoAmazonas_3.9,FrescoAmazonas_3.10
0,1.754,167.01318,NaN,2.102369e+07,5.191424e+06,2.297892e+06,2.022352e+06,1.922735e+06,1.522363e+06,2.475798e+06,...,2.691737e+07,2.667536e+07,3.425704e+06,3.852726e+06,4.906125e+06,4.632700e+06,6.474274e+06,5.309377e+06,1.222893e+07,1.096223e+07
1,2.120,152.05702,NaN,2.420068e+06,2.307995e+06,5.114995e+06,5.131455e+06,5.336355e+06,5.418774e+06,3.795608e+06,...,7.975709e+05,8.330269e+05,1.464225e+06,1.471219e+06,1.590734e+06,1.752319e+06,1.199727e+06,1.253488e+06,1.451674e+06,1.511296e+06
2,2.125,257.96816,NaN,9.001219e+06,9.873734e+06,7.146516e+05,6.834720e+05,7.314678e+06,7.242087e+06,1.013907e+07,...,1.464372e+07,1.561670e+07,1.069788e+06,1.134695e+06,1.007142e+07,9.805008e+06,1.503011e+07,1.535597e+07,1.626041e+07,1.720745e+07
3,2.128,207.98572,NaN,1.938034e+07,2.008965e+07,2.191925e+06,2.174522e+06,5.848345e+06,5.715126e+06,1.004516e+07,...,2.772725e+07,2.676637e+07,2.053523e+06,2.105461e+06,8.143728e+06,7.678579e+06,1.483699e+07,1.450835e+07,1.945829e+07,1.910929e+07
4,2.133,217.96001,NaN,9.418357e+06,9.172249e+06,1.434368e+07,1.480194e+07,1.936935e+07,1.923575e+07,1.545732e+07,...,1.372708e+07,1.383002e+07,2.095277e+07,2.200807e+07,2.830071e+07,2.649514e+07,2.400543e+07,2.153438e+07,2.538244e+07,2.781277e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,20.487,501.26053,NaN,5.709668e+06,5.691558e+06,4.908186e+06,5.048013e+06,5.715906e+06,5.190272e+06,5.309914e+06,...,7.366804e+06,6.834722e+06,6.712807e+06,6.186079e+06,7.730475e+06,7.233852e+06,7.394969e+06,6.816898e+06,7.023647e+06,7.784816e+06
158,22.679,520.34109,NaN,1.034447e+07,1.153674e+07,3.173041e+07,2.880942e+07,2.625058e+07,2.696883e+07,2.167971e+07,...,2.220213e+05,1.821823e+05,9.847894e+05,8.646981e+05,7.707464e+05,7.362852e+05,5.459844e+05,5.837284e+05,6.648591e+05,6.099219e+05
159,23.180,200.20136,NaN,7.190209e+06,6.419463e+06,1.939271e+07,2.036405e+07,2.358837e+07,2.184794e+07,2.493698e+07,...,9.140713e+06,9.683758e+06,3.105293e+07,2.459284e+07,3.208167e+07,3.071925e+07,3.133476e+07,3.285109e+07,4.688143e+07,4.851697e+07
160,24.234,496.34082,NaN,2.836296e+07,2.610888e+07,5.500906e+07,5.810079e+07,3.126886e+07,3.067800e+07,2.151918e+07,...,3.224507e+07,2.965402e+07,4.949670e+07,4.681355e+07,2.604815e+07,2.548679e+07,1.401562e+07,1.510820e+07,1.913652e+07,1.905453e+07


In [28]:
# Classification

print(raw_data_file)
list_metrics = []
for group_id in groups_id + ["Intersection", "Union"]:
    try:
        # Read common node
        common_node_id = np.load(f"experiments/output/{exp}/common_nodes/{group_id}.npy")
        print(group_id, len(common_node_id))
        
        df_join_raw_signal = df_join_raw.loc[common_node_id].iloc[:, 3:]
        df_join_raw_signal_t = df_join_raw_signal.T
        # display(df_join_raw_signal_t)
        
        # Scaling
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(df_join_raw_signal_t.values)
        # print(X_scaled)
        
        # Target
        y = df_join_raw_signal_t.index
        y = [item.split("_")[0] for item in y]
        
        print(X_scaled.shape)
        
        # Split
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
        
        # Select and train model
        rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_classifier.fit(X_train, y_train)

        # Evaluation
        y_pred = rf_classifier.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        recall_macro = recall_score(y_test, y_pred, average="macro")
        f1_macro = f1_score(y_test, y_pred, average="macro")
        # classification_rep = classification_report(y_test, y_pred, output_dict=True)

        # Convert the dictionary to a pandas DataFrame and transpose it
        # df_report = pd.DataFrame(classification_rep).transpose()
        # display(df_report)
        list_metrics.append([group_id, len(common_node_id), accuracy, recall_macro, f1_macro])
    except TypeError as e:
        print(e)
    
df_metrics = pd.DataFrame(list_metrics,  columns=["Group id", "Num. common node", "Accuracy", "Recall", "F1-score"])
df_metrics

deybis_filter_september_2br_3ar_format
SecoAmazonas 81
(160, 81)


FrescoSanMartin 73
(160, 73)
FrescoCusco 110
(160, 110)
FrescoAmazonas 118
(160, 118)
SecoSanMartin 29
(160, 29)
SecoCusco 121
(160, 121)
Intersection 10
(160, 10)
Union 157
(160, 157)


,Group id,Num. common node,Accuracy,Recall,F1-score
0,SecoAmazonas,81,0.81250,0.828571,0.805199
1,FrescoSanMartin,73,0.78125,0.814286,0.810985
2,FrescoCusco,110,0.87500,0.895238,0.887270
3,FrescoAmazonas,118,0.71875,0.738095,0.691880
4,SecoSanMartin,29,0.53125,0.585714,0.522817
5,SecoCusco,121,0.87500,0.885714,0.873329
6,Intersection,10,0.53125,0.566667,0.494672
7,Union,157,0.93750,0.952381,0.951389


In [29]:
# Select best model (dataset and group id)

exp = "exp1" # Change
group_id = "Union" # Change

file = open("experiments/output/{}/parameters.json".format(exp))
params = json.load(file)

print("Exp:\t\t", exp)

raw_data_file = params["raw_data_file"]
print("Raw data:\t", raw_data_file)

# Read common node
common_node_id = np.load(f"experiments/output/{exp}/common_nodes/{group_id}.npy")
print(group_id, len(common_node_id))

df_join_raw_signal = df_join_raw.loc[common_node_id].iloc[:, 3:]
df_join_raw_signal_t = df_join_raw_signal.T
# display(df_join_raw_signal_t)

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_join_raw_signal_t.values)
# print(X_scaled)

# Target
y = df_join_raw_signal_t.index
y = [item.split("_")[0] for item in y]

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Select and train model
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Evaluation
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall_macro = recall_score(y_test, y_pred, average="macro")
f1_macro = f1_score(y_test, y_pred, average="macro")

print("Accuracy:", accuracy)
print("Recall:", recall_macro)
print("F1-score:", f1_macro)

Exp:		 exp1
Raw data:	 deybis_filter_september_2br_3ar_format
Union 157


Accuracy: 0.9375
Recall: 0.9523809523809524
F1-score: 0.951388888888889


Evaluation with December data

In [30]:
# Evaluation with December data

# df_join_raw = pd.read_csv(f"experiments/input/{exp}_raw.csv", index_col=0)
df_raw_new = pd.read_csv(f"experiments/raw_data/deybis_filter_december_all_format.csv", delimiter="|")
df_raw_new

,Alignment ID,Average Rt,Average Mz,Metabolite name,SecoAmazonas_1.1,SecoAmazonas_1.2,SecoAmazonas_1.3,SecoAmazonas_1.4,SecoAmazonas_1.5,SecoAmazonas_1.6,...,FrescoCusco_4.1,FrescoCusco_4.2,FrescoCusco_4.3,FrescoCusco_4.4,FrescoCusco_4.5,FrescoCusco_4.6,FrescoCusco_4.7,FrescoCusco_4.8,FrescoCusco_4.9,FrescoCusco_4.10
0,0,1.754,167.01318,unknown,5.103953e+06,2.909582e+07,3.125169e+07,6.979984e+06,3.776234e+06,2.863133e+07,...,3.014675e+07,3.248538e+07,3.168024e+07,3.168425e+07,5.834778e+06,6.069559e+06,2.108753e+07,4.143768e+06,4.140745e+06,2.110275e+07
1,1,2.120,152.05702,unknown,1.947976e+06,2.145146e+06,1.889418e+06,2.043896e+06,2.002084e+06,2.116984e+06,...,4.518172e+05,4.896768e+05,4.515939e+05,5.737889e+05,4.484360e+05,8.067431e+05,5.232957e+05,6.361791e+05,5.917182e+05,6.042582e+05
2,2,2.125,257.96816,unknown,1.241109e+07,2.402011e+06,2.862010e+06,4.805923e+06,1.147833e+07,3.211076e+06,...,1.073059e+07,1.168772e+07,1.182938e+07,1.109191e+07,4.143309e+06,4.633699e+06,9.860577e+06,1.036408e+07,4.440935e+06,1.046494e+07
3,3,2.128,207.98572,unknown,5.125119e+06,6.072819e+06,5.915750e+06,5.901283e+06,4.214018e+06,5.256240e+06,...,5.687094e+06,3.986381e+06,3.276537e+06,4.797366e+06,4.090073e+06,4.668369e+06,3.019381e+06,3.350595e+06,3.904007e+06,4.440859e+06
4,4,2.133,217.96001,unknown,1.784870e+06,1.576395e+06,9.802001e+05,9.512830e+05,1.182384e+06,1.346597e+06,...,9.267787e+05,1.067227e+06,9.982935e+05,1.098270e+06,1.308553e+06,1.141315e+06,7.890953e+05,8.117531e+05,1.000449e+06,1.090779e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,157,20.487,501.26053,unknown,9.652101e+06,9.339857e+06,8.303287e+06,7.917266e+06,9.801720e+06,7.625165e+06,...,9.011567e+06,8.599511e+06,8.619699e+06,9.089404e+06,7.974695e+06,8.801621e+06,9.053389e+06,8.258463e+06,8.292738e+06,8.316865e+06
158,158,22.679,520.34109,unknown,5.664656e+05,6.374233e+05,5.713218e+05,6.079746e+05,5.665560e+05,6.461613e+05,...,1.536487e+05,1.668626e+05,1.609414e+05,1.613220e+05,1.570790e+05,1.544576e+05,1.391151e+05,1.325859e+05,1.321154e+05,1.297929e+05
159,159,23.180,200.20136,unknown,2.699444e+08,2.478668e+08,2.219785e+08,2.460163e+08,2.424722e+08,2.472737e+08,...,2.782325e+08,2.608633e+08,2.685123e+08,2.508523e+08,2.390475e+08,2.353211e+08,2.752497e+08,2.986559e+08,3.062492e+08,2.700519e+08
160,160,24.234,496.34082,unknown,2.327616e+06,4.292726e+06,4.003384e+06,2.273507e+06,2.707995e+06,3.499644e+06,...,8.344030e+05,1.121508e+06,8.395925e+05,1.199289e+06,9.177739e+05,1.445607e+06,7.684336e+05,1.044961e+06,9.307560e+05,1.094277e+06


In [31]:
# Concat
df_join_raw_new = pd.concat([df_raw_new.iloc[:, :]], axis=1)
df_join_raw_new.set_index("Alignment ID", inplace=True)
df_join_raw_new = df_join_raw_new.rename_axis(None)
df_join_raw_new

,Average Rt,Average Mz,Metabolite name,SecoAmazonas_1.1,SecoAmazonas_1.2,SecoAmazonas_1.3,SecoAmazonas_1.4,SecoAmazonas_1.5,SecoAmazonas_1.6,SecoAmazonas_1.7,...,FrescoCusco_4.1,FrescoCusco_4.2,FrescoCusco_4.3,FrescoCusco_4.4,FrescoCusco_4.5,FrescoCusco_4.6,FrescoCusco_4.7,FrescoCusco_4.8,FrescoCusco_4.9,FrescoCusco_4.10
0,1.754,167.01318,unknown,5.103953e+06,2.909582e+07,3.125169e+07,6.979984e+06,3.776234e+06,2.863133e+07,4.037123e+06,...,3.014675e+07,3.248538e+07,3.168024e+07,3.168425e+07,5.834778e+06,6.069559e+06,2.108753e+07,4.143768e+06,4.140745e+06,2.110275e+07
1,2.120,152.05702,unknown,1.947976e+06,2.145146e+06,1.889418e+06,2.043896e+06,2.002084e+06,2.116984e+06,2.053832e+06,...,4.518172e+05,4.896768e+05,4.515939e+05,5.737889e+05,4.484360e+05,8.067431e+05,5.232957e+05,6.361791e+05,5.917182e+05,6.042582e+05
2,2.125,257.96816,unknown,1.241109e+07,2.402011e+06,2.862010e+06,4.805923e+06,1.147833e+07,3.211076e+06,5.251354e+06,...,1.073059e+07,1.168772e+07,1.182938e+07,1.109191e+07,4.143309e+06,4.633699e+06,9.860577e+06,1.036408e+07,4.440935e+06,1.046494e+07
3,2.128,207.98572,unknown,5.125119e+06,6.072819e+06,5.915750e+06,5.901283e+06,4.214018e+06,5.256240e+06,3.877822e+06,...,5.687094e+06,3.986381e+06,3.276537e+06,4.797366e+06,4.090073e+06,4.668369e+06,3.019381e+06,3.350595e+06,3.904007e+06,4.440859e+06
4,2.133,217.96001,unknown,1.784870e+06,1.576395e+06,9.802001e+05,9.512830e+05,1.182384e+06,1.346597e+06,9.234772e+05,...,9.267787e+05,1.067227e+06,9.982935e+05,1.098270e+06,1.308553e+06,1.141315e+06,7.890953e+05,8.117531e+05,1.000449e+06,1.090779e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,20.487,501.26053,unknown,9.652101e+06,9.339857e+06,8.303287e+06,7.917266e+06,9.801720e+06,7.625165e+06,7.665150e+06,...,9.011567e+06,8.599511e+06,8.619699e+06,9.089404e+06,7.974695e+06,8.801621e+06,9.053389e+06,8.258463e+06,8.292738e+06,8.316865e+06
158,22.679,520.34109,unknown,5.664656e+05,6.374233e+05,5.713218e+05,6.079746e+05,5.665560e+05,6.461613e+05,6.930972e+05,...,1.536487e+05,1.668626e+05,1.609414e+05,1.613220e+05,1.570790e+05,1.544576e+05,1.391151e+05,1.325859e+05,1.321154e+05,1.297929e+05
159,23.180,200.20136,unknown,2.699444e+08,2.478668e+08,2.219785e+08,2.460163e+08,2.424722e+08,2.472737e+08,2.737747e+08,...,2.782325e+08,2.608633e+08,2.685123e+08,2.508523e+08,2.390475e+08,2.353211e+08,2.752497e+08,2.986559e+08,3.062492e+08,2.700519e+08
160,24.234,496.34082,unknown,2.327616e+06,4.292726e+06,4.003384e+06,2.273507e+06,2.707995e+06,3.499644e+06,2.781154e+06,...,8.344030e+05,1.121508e+06,8.395925e+05,1.199289e+06,9.177739e+05,1.445607e+06,7.684336e+05,1.044961e+06,9.307560e+05,1.094277e+06


In [32]:
df_join_raw_signal_new = df_join_raw_new.loc[common_node_id].iloc[:, 3:]
df_join_raw_signal_t_new = df_join_raw_signal_new.T
# display(df_join_raw_signal_t_)

# Scaling
scaler = StandardScaler()
X_scaled_new = scaler.fit_transform(df_join_raw_signal_t_new.values)
# print(X_scaled)

# Target
y_new = df_join_raw_signal_t_new.index
y_new = [item.split("_")[0] for item in y_new]

In [33]:
print(X_scaled_new.shape)
X_scaled_new

(160, 157)


array([[-0.56918262,  0.65074392,  1.69228419, ...,  0.1119761 ,
         0.3462622 ,  0.82679769],
       [ 1.84541068,  0.84372883, -1.03871926, ..., -0.70588462,
         1.77025437,  1.13441589],
       [ 2.0623819 ,  0.59342985, -0.91320745, ..., -1.6649185 ,
         1.56058672,  0.84298994],
       ...,
       [-0.66581767, -0.63320329,  1.13375297, ...,  1.17559219,
        -0.5831981 , -0.81736482],
       [-0.66612197, -0.6767203 , -0.48239343, ...,  1.45688755,
        -0.66595518, -0.81857676],
       [ 1.04097044, -0.66444646,  1.16127198, ...,  0.1159606 ,
        -0.54746159, -0.81379584]])

In [34]:
print(len(y_new))
y_new

160


['SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoCusco',
 'SecoCusco',
 'SecoCusco',
 'SecoCusco',
 'SecoCusco',
 'SecoCusco',
 'SecoCusco',
 'SecoCusco',
 'SecoCusco',
 'Se

In [35]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_scaled_new, y_new, test_size=0.1, random_state=42)
print(len(y_train_new))
y_train_new

144


['FrescoCusco',
 'FrescoAmazonas',
 'FrescoSanMartin',
 'SecoAmazonas',
 'FrescoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoSanMartin',
 'FrescoCusco',
 'FrescoSanMartin',
 'SecoCusco',
 'FrescoCusco',
 'FrescoSanMartin',
 'FrescoCusco',
 'FrescoAmazonas',
 'SecoCusco',
 'FrescoCusco',
 'SecoAmazonas',
 'FrescoCusco',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoCusco',
 'FrescoSanMartin',
 'SecoAmazonas',
 'FrescoCusco',
 'FrescoAmazonas',
 'FrescoSanMartin',
 'FrescoAmazonas',
 'SecoSanMartin',
 'FrescoAmazonas',
 'SecoAmazonas',
 'FrescoAmazonas',
 'SecoCusco',
 'FrescoCusco',
 'SecoAmazonas',
 'SecoCusco',
 'FrescoAmazonas',
 'SecoAmazonas',
 'SecoAmazonas',
 'FrescoAmazonas',
 'FrescoCusco',
 'FrescoCusco',
 'SecoSanMartin',
 'SecoAmazonas',
 'FrescoCusco',
 'SecoSanMartin',
 'FrescoCusco',
 'FrescoCusco',
 'SecoCusco',
 'SecoAmazonas',
 'FrescoSanMartin',
 'FrescoSanMartin',
 'SecoAmazonas',
 'FrescoCusco',
 'FrescoCusco',
 'SecoCusco',
 'SecoSanMartin',
 'Fr

In [36]:
y_pred_new = rf_classifier.predict(X_train_new)
print(len(y_pred_new))
y_pred_new.tolist()

144


['FrescoCusco',
 'FrescoAmazonas',
 'FrescoCusco',
 'SecoAmazonas',
 'FrescoCusco',
 'SecoAmazonas',
 'SecoAmazonas',
 'SecoSanMartin',
 'SecoSanMartin',
 'FrescoCusco',
 'FrescoCusco',
 'SecoCusco',
 'FrescoCusco',
 'FrescoCusco',
 'FrescoCusco',
 'FrescoCusco',
 'SecoCusco',
 'FrescoCusco',
 'SecoAmazonas',
 'FrescoCusco',
 'SecoSanMartin',
 'SecoSanMartin',
 'SecoCusco',
 'FrescoSanMartín',
 'SecoAmazonas',
 'FrescoCusco',
 'FrescoCusco',
 'FrescoCusco',
 'FrescoAmazonas',
 'SecoSanMartin',
 'FrescoAmazonas',
 'SecoAmazonas',
 'FrescoAmazonas',
 'SecoCusco',
 'FrescoCusco',
 'SecoAmazonas',
 'SecoCusco',
 'FrescoCusco',
 'SecoAmazonas',
 'SecoAmazonas',
 'FrescoAmazonas',
 'FrescoCusco',
 'FrescoCusco',
 'SecoSanMartin',
 'SecoAmazonas',
 'FrescoCusco',
 'SecoSanMartin',
 'FrescoCusco',
 'FrescoCusco',
 'SecoCusco',
 'SecoAmazonas',
 'FrescoCusco',
 'FrescoCusco',
 'SecoAmazonas',
 'FrescoCusco',
 'FrescoCusco',
 'SecoCusco',
 'SecoSanMartin',
 'FrescoAmazonas',
 'SecoAmazonas',
 'S

In [37]:
accuracy = accuracy_score(y_train_new, y_pred_new)
recall_macro = recall_score(y_train_new, y_pred_new, average="macro")
f1_macro = f1_score(y_train_new, y_pred_new, average="macro")

print("Accuracy:", accuracy)
print("Recall:", recall_macro)
print("F1-score:", f1_macro)

Accuracy: 0.7916666666666666
Recall: 0.6322858428121586
F1-score: 0.6252102015836936


/home/ealvarez/miniconda3/envs/graph_matching/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
df_comp = pd.DataFrame({
    "y true": y_train_new,
    "y pred": y_pred_new
})
df_comp

,y true,y pred
0,FrescoCusco,FrescoCusco
1,FrescoAmazonas,FrescoAmazonas
2,FrescoSanMartin,FrescoCusco
3,SecoAmazonas,SecoAmazonas
4,FrescoAmazonas,FrescoCusco
...,...,...
139,FrescoAmazonas,FrescoCusco
140,FrescoSanMartin,FrescoSanMartín
141,SecoAmazonas,SecoAmazonas
142,FrescoSanMartin,FrescoSanMartín
